In [1]:
import os
from datetime import date
import pandas as pd
import itertools
import empyrical as empy
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings("ignore")

In [2]:
def drawdown_from_price(price):
    if isinstance(price, pd.Series):
        return _drawdown_from_price(price)
    elif isinstance(price, pd.DataFrame):
        return price.apply(_drawdown_from_price)
    else:
        raise non_pd_error

def _drawdown_from_price(price):
    shift_max = price.copy()
    _max = price.iloc[0]
    for i, j in price.items():
        #print('i', i)
        #print('j', j)
        _max = max(_max, j)
        shift_max[i] = _max
    return price / shift_max - 1

def return_to_price(ret, ini=100):
    price_0 = ret.dropna().iloc[:1] * 0 + ini
    price_0.index = [0]
    price = (1+ret).cumprod() * ini
    return pd.concat([price_0, price])

def drawdown_from_return(ret, ini=100):
    price = return_to_price(ret, ini)
    return drawdown_from_price(price).iloc[1:]

def avg_drawdown(ret):
    dd = drawdown_from_return(ret)
    return dd.mean()

def empy_metric(ret):
    if isinstance(ret, pd.DataFrame):
        return ret.apply(empy_metric).T
    total_return = lambda x: (1+x).prod()-1
    met_func = [
        total_return, 
        lambda x: empy.annual_return(x), 
        lambda x: empy.sharpe_ratio(x), 
        lambda x: empy.annual_volatility(x), 
        lambda x: empy.max_drawdown(x), 
        avg_drawdown]
    
    met_func_names = ['total_return', 'annual_return', 'sharpe_ratio', 'annual_volatility', 
                      'max_drawdown', 'avg_drawdown',]
    
    se = pd.Series([f(ret) for f in met_func], met_func_names)
    
    se['return/maxdd'] = -se.annual_return/se.max_drawdown
    se['return/avgdd'] = -se.annual_return/se.avg_drawdown
    
    buy01 = ret.apply(lambda x: 0 if x==0 else 1)
    se['buy_ratio'] = buy01.mean()
    se['flip_ratio'] = (buy01-buy01.shift()).abs().mean()

    return se

In [3]:
client_weight_agg = {'sharpe_ratio':5,'annual_return':5, 'max_drawdown':5}
def calculation_score(df, client_weight=client_weight_agg):
    df['Sharpe Score'] = df['sharpe_ratio'].apply(lambda x: x if x>0 else 0) * 100
    df['Return Score'] = df['annual_return'].apply(lambda x: 1+x/2) * 100
    df['Max-DD Score'] = df['max_drawdown'].apply(lambda x: 1+x) * 100

    df['Total Score'] = 0
    for score, weight in client_weight.items():
        df['Total Score'] += weight * df[score]
    return df

In [4]:
def display_df(df, export_img=False, **kwargs):
    df_style = df.style.background_gradient(**kwargs)
    display(df_style)
    
    if export_img:
        try:
            dfi.export(df_style,"tmp.png",table_conversion='matplotlib')
            display(Image("tmp.png"))
        except:
            pass

def show_cum_ret(ret):
    fig, ax = plt.subplots(figsize=(24,15))
    cm = plt.get_cmap('gist_rainbow')
    NUM_COLORS = ret.shape[1]
    ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])

    (1+ret).cumprod().plot(ax=ax)

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_yscale("log")
    plt.show()

In [5]:
ret_list = [
    './result/2022/cashON_signal/result/ret_slding.csv',
    # './result/2022/cashON_signal/result/ret_extending.csv'
]

In [26]:
ret_cases = pd.DataFrame()
for path in ret_list:
    df = pd.read_csv(path, index_col=0)
    df.index = pd.to_datetime(df.index)
    ret_cases = pd.concat([ret_cases, df], axis=1)
ret_cases

,lookback1Y_sortbyTotal Score,lookback1Y_sortbysharpe_ratio,lookback1Y_sortbyreturn/avgdd,lookback1Y_sortbymax_drawdown,lookback1Y_sortbyavg_drawdown,lookback2Y_sortbyTotal Score,lookback2Y_sortbysharpe_ratio,lookback2Y_sortbyreturn/avgdd,lookback2Y_sortbymax_drawdown,lookback2Y_sortbyavg_drawdown,...,lookback4Y_sortbysharpe_ratio,lookback4Y_sortbyreturn/avgdd,lookback4Y_sortbymax_drawdown,lookback4Y_sortbyavg_drawdown,lookback5Y_sortbyTotal Score,lookback5Y_sortbysharpe_ratio,lookback5Y_sortbyreturn/avgdd,lookback5Y_sortbymax_drawdown,lookback5Y_sortbyavg_drawdown,no_3vote
2008-01-02,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,...,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031
2008-01-03,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,...,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646
2008-01-04,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,...,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659
2008-01-07,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,...,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824
2008-01-08,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,...,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-25,-0.000582,-0.000582,-0.000582,-0.000582,-0.000582,-0.000582,-0.000582,-0.000582,-0.000582,-0.000582,...,-0.000582,-0.000582,-0.000582,-0.000582,-0.000582,-0.000582,-0.000582,-0.000582,-0.000582,-0.000582
2022-07-26,-0.002362,-0.002362,-0.002362,-0.002362,-0.002362,-0.002362,-0.002362,-0.002362,-0.002362,-0.002362,...,-0.002362,-0.002362,-0.002362,-0.002362,-0.002362,-0.002362,-0.002362,-0.002362,-0.002362,-0.002362
2022-07-27,0.009400,0.009400,0.009400,0.009400,0.009400,0.009400,0.009400,0.009400,0.009400,0.009400,...,0.009400,0.009400,0.009400,0.009400,0.009400,0.009400,0.009400,0.009400,0.009400,0.009400
2022-07-28,0.009597,0.009597,0.009597,0.009597,0.009597,0.009597,0.009597,0.009597,0.009597,-0.000101,...,0.009597,0.009597,-0.000101,-0.000101,0.009597,0.009597,-0.000101,-0.000101,0.009597,0.009597


In [27]:
no_3vote_ret = ret_cases['no_3vote']

In [28]:
ret_cases = ret_cases.drop('no_3vote', axis=1)
ret_cases_stage = (ret_cases!=0).astype(int)
ret_cases_stage

,lookback1Y_sortbyTotal Score,lookback1Y_sortbysharpe_ratio,lookback1Y_sortbyreturn/avgdd,lookback1Y_sortbymax_drawdown,lookback1Y_sortbyavg_drawdown,lookback2Y_sortbyTotal Score,lookback2Y_sortbysharpe_ratio,lookback2Y_sortbyreturn/avgdd,lookback2Y_sortbymax_drawdown,lookback2Y_sortbyavg_drawdown,...,lookback4Y_sortbyTotal Score,lookback4Y_sortbysharpe_ratio,lookback4Y_sortbyreturn/avgdd,lookback4Y_sortbymax_drawdown,lookback4Y_sortbyavg_drawdown,lookback5Y_sortbyTotal Score,lookback5Y_sortbysharpe_ratio,lookback5Y_sortbyreturn/avgdd,lookback5Y_sortbymax_drawdown,lookback5Y_sortbyavg_drawdown
2008-01-02,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2008-01-03,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2008-01-04,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2008-01-07,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2008-01-08,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-25,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2022-07-26,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2022-07-27,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2022-07-28,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [7]:
mly_list = [
    ('./result/2022/202205v0_till202208/3voteContNdays_(LQD,TIP,VTI)202205v0_till202208.csv', 'vote(LQD,TIP,VTI)'),
    ('./result/2022/202205v0_till202208/5voteContNdays_202205v0_till202208.csv', 'vote(GLD,TLT,LQD,TIP,VTI)'),
    ('./result/2022/202205v0_till202208/3voteContNdays_202205v0_till202208.csv', 'vote(GLD,TLT,TIP)'),
]

In [8]:
replace_str = 'trans_preds_202205v0_till202208'
preds = pd.DataFrame()
for path, name in mly_list:
    df = pd.read_csv(path, index_col=0)
    df.index = pd.to_datetime(df.index)
    df.columns = [col.replace(replace_str, name) for col in df.columns]
    preds = pd.concat([preds, df], axis=1)
preds

,"vote(LQD,TIP,VTI)___freqW","vote(LQD,TIP,VTI)___freqM","vote(LQD,TIP,VTI)___cont1_rb_lookback","vote(LQD,TIP,VTI)___cont1_rb_follow","vote(LQD,TIP,VTI)___cont1quit_cont2back_rb_lookback","vote(LQD,TIP,VTI)___cont1quit_cont2back_rb_follow","vote(LQD,TIP,VTI)___cont1quit_cont3back_rb_lookback","vote(LQD,TIP,VTI)___cont1quit_cont3back_rb_follow","vote(LQD,TIP,VTI)___cont1quit_cont4back_rb_lookback","vote(LQD,TIP,VTI)___cont1quit_cont4back_rb_follow",...,"vote(GLD,TLT,TIP)___cont3quit_cont4back_rb_lookback","vote(GLD,TLT,TIP)___cont3quit_cont4back_rb_follow","vote(GLD,TLT,TIP)___cont4quit_cont1back_rb_lookback","vote(GLD,TLT,TIP)___cont4quit_cont1back_rb_follow","vote(GLD,TLT,TIP)___cont4quit_cont2back_rb_lookback","vote(GLD,TLT,TIP)___cont4quit_cont2back_rb_follow","vote(GLD,TLT,TIP)___cont4quit_cont3back_rb_lookback","vote(GLD,TLT,TIP)___cont4quit_cont3back_rb_follow","vote(GLD,TLT,TIP)___cont4_rb_lookback","vote(GLD,TLT,TIP)___cont4_rb_follow"
Trading Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2008-01-03,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2008-01-04,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2008-01-07,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2008-01-08,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-08-01,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-08-02,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
pairs = list(itertools.product(list(ret_cases_stage.columns), list(preds.columns)))
len(pairs)

2550

In [23]:
combine_cases = pd.DataFrame()
exists_case = []
for a, b in pairs:
    # print(a, b)
    if a == b:
        continue

    if (a, b) in exists_case or (b, a) in exists_case:
        continue

    tmp = pd.concat([ret_cases_stage[a], preds[b]], axis=1).astype(bool)
    # display(tmp)

    name = "({})&({})".format(a, b)
    combine_cases[name] = tmp[a] & tmp[b]

    name = "({})|({})".format(a, b)
    combine_cases[name] = tmp[a] | tmp[b]

    exists_case += [(a, b), (b, a)]
    # display(vote_cases)

    # display(exists_case)

    # if len(vote_cases.columns) >= 4:
    #     break
combine_cases = combine_cases.astype(int)

In [32]:
combine_cases['no_3vote'] = 1
all_kinds_return = combine_cases.apply(lambda x: x*no_3vote_ret).dropna()
all_kinds_return

,"(lookback1Y_sortbyTotal Score)&(vote(LQD,TIP,VTI)___freqW)","(lookback1Y_sortbyTotal Score)|(vote(LQD,TIP,VTI)___freqW)","(lookback1Y_sortbyTotal Score)&(vote(LQD,TIP,VTI)___freqM)","(lookback1Y_sortbyTotal Score)|(vote(LQD,TIP,VTI)___freqM)","(lookback1Y_sortbyTotal Score)&(vote(LQD,TIP,VTI)___cont1_rb_lookback)","(lookback1Y_sortbyTotal Score)|(vote(LQD,TIP,VTI)___cont1_rb_lookback)","(lookback1Y_sortbyTotal Score)&(vote(LQD,TIP,VTI)___cont1_rb_follow)","(lookback1Y_sortbyTotal Score)|(vote(LQD,TIP,VTI)___cont1_rb_follow)","(lookback1Y_sortbyTotal Score)&(vote(LQD,TIP,VTI)___cont1quit_cont2back_rb_lookback)","(lookback1Y_sortbyTotal Score)|(vote(LQD,TIP,VTI)___cont1quit_cont2back_rb_lookback)",...,"(lookback5Y_sortbyavg_drawdown)|(vote(GLD,TLT,TIP)___cont4quit_cont2back_rb_follow)","(lookback5Y_sortbyavg_drawdown)&(vote(GLD,TLT,TIP)___cont4quit_cont3back_rb_lookback)","(lookback5Y_sortbyavg_drawdown)|(vote(GLD,TLT,TIP)___cont4quit_cont3back_rb_lookback)","(lookback5Y_sortbyavg_drawdown)&(vote(GLD,TLT,TIP)___cont4quit_cont3back_rb_follow)","(lookback5Y_sortbyavg_drawdown)|(vote(GLD,TLT,TIP)___cont4quit_cont3back_rb_follow)","(lookback5Y_sortbyavg_drawdown)&(vote(GLD,TLT,TIP)___cont4_rb_lookback)","(lookback5Y_sortbyavg_drawdown)|(vote(GLD,TLT,TIP)___cont4_rb_lookback)","(lookback5Y_sortbyavg_drawdown)&(vote(GLD,TLT,TIP)___cont4_rb_follow)","(lookback5Y_sortbyavg_drawdown)|(vote(GLD,TLT,TIP)___cont4_rb_follow)",no_3vote
2008-01-02,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,...,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031,0.021031
2008-01-03,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,...,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646,0.004646
2008-01-04,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,...,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659,-0.008659
2008-01-07,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,...,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824,-0.003824
2008-01-08,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,...,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096,0.002096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-25,-0.000000,-0.000582,-0.000000,-0.000582,-0.000000,-0.000582,-0.000000,-0.000582,-0.000000,-0.000582,...,-0.000582,-0.000000,-0.000582,-0.000000,-0.000582,-0.000000,-0.000582,-0.000000,-0.000582,-0.000582
2022-07-26,-0.000000,-0.002362,-0.000000,-0.002362,-0.000000,-0.002362,-0.000000,-0.002362,-0.000000,-0.002362,...,-0.002362,-0.000000,-0.002362,-0.000000,-0.002362,-0.000000,-0.002362,-0.000000,-0.002362,-0.002362
2022-07-27,0.000000,0.009400,0.000000,0.009400,0.000000,0.009400,0.000000,0.009400,0.000000,0.009400,...,0.009400,0.000000,0.009400,0.000000,0.009400,0.000000,0.009400,0.000000,0.009400,0.009400
2022-07-28,0.000000,0.009597,0.000000,0.009597,0.000000,0.009597,0.000000,0.009597,0.000000,0.009597,...,0.009597,0.000000,0.009597,0.000000,0.009597,0.000000,0.009597,0.000000,0.009597,0.009597


In [ ]:
met = empy_metric(all_kinds_return)
met = calculation_score(met)
met.sort_values('sharpe_ratio', ascending=False)[:'no_3vote']

In [34]:
met.sort_values('sharpe_ratio', ascending=False)[:'no_3vote']

,total_return,annual_return,sharpe_ratio,annual_volatility,max_drawdown,avg_drawdown,return/maxdd,return/avgdd,buy_ratio,flip_ratio,Sharpe Score,Return Score,Max-DD Score,Total Score
"(lookback5Y_sortbymax_drawdown)|(vote(GLD,TLT,LQD,TIP,VTI)___cont1quit_cont4back_rb_lookback)",3.825893,0.114135,0.911021,0.127604,-0.206512,-0.045252,0.552680,2.522213,0.951771,0.018534,91.102075,105.706742,79.348844,4.093220
"(lookback5Y_sortbymax_drawdown)|(vote(GLD,TLT,LQD,TIP,VTI)___cont1quit_cont4back_rb_follow)",3.790318,0.113569,0.906577,0.127680,-0.206512,-0.045571,0.549940,2.492145,0.953951,0.016353,90.657705,105.678448,79.348844,4.068172
"(lookback3Y_sortbymax_drawdown)|(vote(GLD,TLT,LQD,TIP,VTI)___cont1quit_cont4back_rb_lookback)",3.826692,0.114148,0.898226,0.129742,-0.216326,-0.050523,0.527665,2.259311,0.961035,0.020169,89.822580,105.707376,78.367417,3.980237
"(lookback3Y_sortbymax_drawdown)|(vote(GLD,TLT,LQD,TIP,VTI)___cont1quit_cont4back_rb_follow)",3.777164,0.113359,0.892665,0.129759,-0.219639,-0.051395,0.516114,2.205623,0.961580,0.019079,89.266491,105.667936,78.036111,3.931924
"(lookback5Y_sortbymax_drawdown)|(vote(GLD,TLT,TIP)___cont1quit_cont4back_rb_follow)",3.572098,0.110010,0.885945,0.126931,-0.206512,-0.046093,0.532704,2.386663,0.944959,0.020169,88.594527,105.500480,79.348844,3.947216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(lookback1Y_sortbyavg_drawdown)|(vote(GLD,TLT,TIP)___cont1quit_cont4back_rb_follow)",2.949797,0.098914,0.789918,0.130168,-0.222995,-0.054774,0.443570,1.805866,0.962943,0.021259,78.991822,104.945702,77.700450,3.329184
"(lookback1Y_sortbyreturn/avgdd)&(vote(GLD,TLT,LQD,TIP,VTI)___freqM)",2.275495,0.084879,0.789889,0.110954,-0.157565,-0.039730,0.538692,2.136401,0.750409,0.035159,78.988940,104.243964,84.243461,3.586016
"(lookback4Y_sortbymax_drawdown)|(vote(LQD,TIP,VTI)___freqM)",2.916534,0.098276,0.789857,0.129300,-0.220749,-0.054002,0.445194,1.819850,0.972207,0.008177,78.985708,104.913803,77.925125,3.336922
"(lookback4Y_sortbysharpe_ratio)|(vote(GLD,TLT,LQD,TIP,VTI)___cont2quit_cont4back_rb_lookback)",2.926330,0.098464,0.789712,0.129590,-0.206589,-0.052797,0.476620,1.864953,0.968120,0.014718,78.971174,104.923224,79.341105,3.407936


In [35]:
all_kinds_return.to_csv('./result/2022/cashON_signal/result/ret_slding&ML.csv')
met.to_csv('./result/2022/cashON_signal/result/met_slding&ML.csv')